In [11]:
# original tutorial: https://docs.pymc.io/notebooks/api_quickstart.html

In [2]:

%matplotlib inline
import numpy as np
import theano.tensor as tt
import pymc3 as pm

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_context('notebook')
plt.style.use('seaborn-darkgrid')
print('Running on PyMC3 v{}'.format(pm.__version__))

/Users/m102417/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Running on PyMC3 v3.7


# Model creation

Models in PyMC3 are centered around the Model class. It has references to all random variables (RVs) and computes the model logp and its gradients. Usually, you would instantiate it as part of a with context:

In [3]:
with pm.Model() as model:
    # Model definition
    pass

We discuss RVs further below but let’s create a simple model to explore the Model class.

In [4]:
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=1, observed=np.random.randn(100))


In [5]:
model.basic_RVs

[mu, obs]

In [6]:
model.free_RVs

[mu]

In [7]:
model.observed_RVs

[obs]

In [8]:
model.logp({'mu': 0})

array(-143.08159657)

Warning It’s worth highlighting one of the counter-intuitive design choices with logp. The API makes the logp look like an attribute, when it actually puts together a function based on the current state of the model.

The current design is super maintainable, does terrible if the state stays constant, and great if the state keeps changing, for reasons of design we assume that Model isn’t static, in fact it’s best in our experience and avoids bad results.

If you need to use logp in an inner loop and it needs to be static, simply use something like logp = model.logp below. You can see the caching effect with the speed up below.

In [9]:
%timeit model.logp({mu: 0.1})
logp = model.logp
%timeit logp({mu: 0.1})

59.1 ms ± 2.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
17.8 µs ± 1.43 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Probability Distributions¶
Every probabilistic program consists of observed and unobserved Random Variables (RVs). Observed RVs are defined via likelihood distributions, while unobserved RVs are defined via prior distributions. In PyMC3, probability distributions are available from the main module space:

In [10]:

help(pm.Normal)

Help on class Normal in module pymc3.distributions.continuous:

class Normal(pymc3.distributions.distribution.Continuous)
 |  Univariate normal log-likelihood.
 |  
 |  The pdf of this distribution is
 |  
 |  .. math::
 |  
 |     f(x \mid \mu, \tau) =
 |         \sqrt{\frac{\tau}{2\pi}}
 |         \exp\left\{ -\frac{\tau}{2} (x-\mu)^2 \right\}
 |  
 |  Normal distribution can be parameterized either in terms of precision
 |  or standard deviation. The link between the two parametrizations is
 |  given by
 |  
 |  .. math::
 |  
 |     \tau = \dfrac{1}{\sigma^2}
 |  
 |  .. plot::
 |  
 |      import matplotlib.pyplot as plt
 |      import numpy as np
 |      import scipy.stats as st
 |      plt.style.use('seaborn-darkgrid')
 |      x = np.linspace(-5, 5, 1000)
 |      mus = [0., 0., 0., -2.]
 |      sigmas = [0.4, 1., 2., 0.4]
 |      for mu, sigma in zip(mus, sigmas):
 |          pdf = st.norm.pdf(x, mu, sigma)
 |          plt.plot(x, pdf, label=r'$\mu$ = {}, $\sigma$ = {}'.format(m

In the PyMC3 module, the structure for probability distributions looks like this:

pymc3.distributions - https://docs.pymc.io/api/distributions.html
* continuous - https://docs.pymc.io/api/distributions/continuous.html
* discrete - https://docs.pymc.io/api/distributions/discrete.html
* timeseries - https://docs.pymc.io/api/distributions/timeseries.html
* mixture - https://docs.pymc.io/api/distributions/mixture.html

In [11]:
dir(pm.distributions.mixture)

['Discrete',
 'Distribution',
 'Iterable',
 'Mixture',
 'Normal',
 'NormalMixture',
 '_DrawValuesContext',
 '_DrawValuesContextBlocker',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_conversion_map',
 'all_discrete',
 'bound',
 'broadcast_distribution_samples',
 'draw_values',
 'generate_samples',
 'get_tau_sigma',
 'get_variable_name',
 'logsumexp',
 'np',
 'random_choice',
 'theano',
 'to_tuple',
 'tt']

# Unobserved Random Variables¶
Every unobserved RV has the following calling signature: name (str), parameter keyword arguments. Thus, a normal prior can be defined in a model context like this:



In [12]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1)

As with the model, we can evaluate its logp:

In [13]:

x.logp({'x': 0})

array(-0.91893853)

# Observed Random Variables¶
Observed RVs are defined just like unobserved RVs but require data to be passed into the observed keyword argument:

In [14]:
with pm.Model():
    obs = pm.Normal('x', mu=0, sigma=1, observed=np.random.randn(100))

observed supports lists, numpy.ndarray, theano and pandas data structures.

# Deterministic transforms¶
PyMC3 allows you to freely do algebra with RVs in all kinds of ways:

In [15]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1)
    y = pm.Gamma('y', alpha=1, beta=1)
    plus_2 = x + 2
    summed = x + y
    squared = x**2
    sined = pm.math.sin(x)

While these transformations work seamlessly, their results are not stored automatically. Thus, if you want to keep track of a transformed variable, you have to use pm.Deterministic:

In [16]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1)
    plus_2 = pm.Deterministic('x plus 2', x + 2)

Note that plus_2 can be used in the identical way to above, we only tell PyMC3 to keep track of this RV for us.

# Automatic transforms of bounded RVs¶
In order to sample models more efficiently, PyMC3 automatically transforms bounded RVs to be unbounded.

In [18]:
with pm.Model() as model:
    x = pm.Uniform('x', lower=0, upper=1)

When we look at the RVs of the model, we would expect to find x there, however:

In [19]:
model.free_RVs

[x_interval__]

x_interval__ represents x transformed to accept parameter values between -inf and +inf. In the case of an upper and a lower bound, a LogOdds transform is applied. Sampling in this transformed space makes it easier for the sampler. PyMC3 also keeps track of the non-transformed, bounded parameters. These are common determinstics (see above):